In [ ]:
import grass.script as gs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Define raster years ===
years = [1985, 1990, 1995, 2000] + list(range(2001, 2023))
raster_prefix = "GLC_FCS30_"

# === 2. Class label mapping (from your rasters) ===
class_labels = {
    5: "Shrubland",
    7: "Grassland",
    8: "Cropland",
    9: "Wetland",
    11: "Lichens and Mosses",
    12: "Bareland",
    13: "Built-up",
    15: "Water",
    16: "Permanent ice and snow",
    20: "Forest",
}

# === 3. Collect area stats from each raster ===
all_stats = []

for year in years:
    raster = f"{raster_prefix}{year}"
    print(f"📊 Processing {raster}...")

    try:
        # Check raster presence without mapset suffix
        available_rasters = [
            r.split("@")[0] for r in gs.read_command("g.list", type="raster", flags="m").splitlines()
        ]

        if raster not in available_rasters:
            print(f"⚠️ Raster {raster} not found, skipping.")
            continue

        # Set region to match raster
        gs.run_command("g.region", raster=raster, quiet=True)

        # Extract class statistics (value;area)
        stats = gs.read_command("r.stats", flags="an", input=raster, sep=";", quiet=True)

        for line in stats.strip().split("\n"):
            parts = line.strip().split(";")
            if len(parts) != 2:
                continue

            try:
                val = int(parts[0])
                area_m2 = float(parts[1])
                area_km2 = area_m2 / 1e6  # Convert to square kilometers

                if val in class_labels:
                    all_stats.append({
                        "Year": year,
                        "Class": class_labels[val],
                        "Area_km2": area_km2
                    })

            except Exception as e:
                print(f"⚠️ Parse error in {raster}: {line} → {e}")
                continue

    except Exception as e:
        print(f"❌ Error with {raster}: {e}")
        continue

# === 4. Create DataFrame & calculate percentages ===
df = pd.DataFrame(all_stats)

if df.empty:
    raise ValueError("No stats collected. Check raster availability and class values.")

# Calculate total area and percentage for each class in each year
def calculate_stats(group):
    total_area = group['Area_km2'].sum()
    group['pct'] = (group['Area_km2'] / total_area) * 100
    group['count'] = (group['Area_km2'] * 1e6 / 90)  # Assuming 90m resolution for pixel count
    return group

df = df.groupby('Year').apply(calculate_stats)

# Format the output similarly to your sample
def format_stats(year_data):
    year_data = year_data.sort_values('Area_km2', ascending=False)
    year_data['Area_km2'] = year_data['Area_km2'].map('{:,.1f}'.format)
    year_data['pct'] = year_data['pct'].map('{:.2f}%'.format)
    year_data['count'] = year_data['count'].map('{:,.0f}'.format)
    return year_data

# Print sample statistics for each year
for year in years:
    year_data = df[df['Year'] == year]
    if not year_data.empty:
        formatted = format_stats(year_data.copy())
        print(f"\nSample Statistics for {year}:")
        print(formatted[['Class', 'count', 'Area_km2', 'pct']].to_string(index=False))

# === 5. Pivot table for time-series plotting ===
df_pivot = df.pivot(index="Year", columns="Class", values="Area_km2").fillna(0)

# === 6. Plot line chart of land cover over time ===
sns.set(style="whitegrid", font_scale=1.3)
plt.figure(figsize=(14, 7))

for col in df_pivot.columns:
    plt.plot(df_pivot.index, df_pivot[col], label=col, linewidth=2.2)

plt.title("🌍 Land Cover Dynamics (1985–2022)", fontsize=16)
plt.xlabel("Year")
plt.ylabel("Area (Square km)")
plt.legend(title="Land Cover Type", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True)
plt.tight_layout()
plt.show()

# === 7. Save to CSV ===
df.to_csv("land_cover_stats.csv", index=False)
print("✅ land_cover_stats.csv saved.")


